In [ ]:
# importa macro %%cu
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

# carrega plugin
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1zlycxbb
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1zlycxbb
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=0da40964510f46abd7392a74fd30473c896cc454d37880c93bdb31a82ffdf2fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-i0qlvqak/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%cd /usr/local
%cd cuda-10.0
%cd samples
%cd 1_Utilities
%cd deviceQuery
!ls
!make
!./deviceQuery

/usr/local
/usr/local/cuda-10.0
/usr/local/cuda-10.0/samples
/usr/local/cuda-10.0/samples/1_Utilities
/usr/local/cuda-10.0/samples/1_Utilities/deviceQuery
deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt
/usr/local/cuda-10.0/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda-10.0/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch

In [20]:
# %%cu
%%writefile id.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <unistd.h>
#include <assert.h>
#include <sys/time.h>

//Função para calcular MDC
__device__ int gcd(int u, int v) {
	if (v == 0)
		return u;
	return gcd(v, u % v);
}

__global__ void soma_divisores(long int*g_sequencia, long int *g_num, long int *g_den, int tam){
    int pos,sum, factor, done,n;

    //Pega a posição do vetor que aquela thread irá trabalhar
    pos = (blockIdx.x*blockDim.x)+threadIdx.x;

    //Verifica se está em uma posição válida, uma vez que o último bloco
    //pode ter threads restantes, ou seja, o posição pode exceder o número de elementos
    if(pos<tam){
        
        //Já soma o próprio elemento e 1, pois são divisores
        sum = 1 + g_sequencia[pos];
        done = g_sequencia[pos];
        factor = 2;

        while (factor < (done/2)+1) {  // faz a soma dos elementos divisiveis por i
          if ((g_sequencia[pos] % factor) == 0) {
            sum += (factor);
          }
          factor++;
        }
        g_num[pos] = sum; 
        g_den[pos] = g_sequencia[pos]; 
        //Faz o MDC pra encontrar o numerador e o denominador
        n = gcd(g_num[pos], g_den[pos]); 
        g_num[pos] /= n; //numerador
        g_den[pos] /= n; //denominador
        
    }
        
}

int main(int argc, char **argv) {
	long int start;
	long int end,n;
  int i, count,j;
	struct timeval inic,fim;

	while (1) {
		scanf("%ld %ld", &start, &end);
		if (start == 0 && end == 0)
			break;
    
	  gettimeofday(&inic,0);

    //calcula a quantidade de elementos
    long int last = end - start + 1;
		printf("Number %ld to %ld\n", start, end);
    
    //aloca um vetor no device e outro no host para armazenar todos os elementos
    long int *sequencia, *g_sequencia;
    sequencia = (long int*) malloc(sizeof(long int) * last);
    cudaMalloc(&g_sequencia, sizeof(long int)*last);

    //Inicializa o vetor de sequencias
    for (i = start,count = 0; i <= end; i++,count++) {
        sequencia[count] = i;
    }

    //copia as sequencias do host pra GPU
    cudaMemcpy(g_sequencia, sequencia, sizeof(long int)*last, cudaMemcpyHostToDevice);
   
    //aloca o vetor de numerador na GPU e host
    long int *num, *g_num;
    num = (long int*) malloc(sizeof(long int) * last);
    cudaMalloc(&g_num, sizeof(long int)*last);

    //aloca o vetor de denominador na GPU e host
    long int *den, *g_den;
    den = (long int*) malloc(sizeof(long int) * last);
    cudaMalloc(&g_den, sizeof(long int)*last);

    int threadsPerBlock;
    //se o número de elementos for menor que 1024, define 
    //apenas 1 bloco com a quantidade de threads igual a de elementos
    if(last<32){
        n=1;
        threadsPerBlock = last;
    //Se não, verifica o número de blocos que será necessário
    }else{
        n = last/32;
        if(last%32 != 0){ // Caso o número de blocos não seja divisível por 1024
          n+=1;             //adiciona-se mais um bloco, que recebará a última parte
        }                   //menor que 1024 
        threadsPerBlock = 32;
    }
    
    int dimGrid = n;

    //chama o kernel
    soma_divisores<<<dimGrid, threadsPerBlock>>>(g_sequencia, g_num, g_den, last);

    //Função para o Host aguardar a execução de todas as threads
    cudaDeviceSynchronize();
    
    //copia os vetores de numerador e denomiador da GPU pro host
    cudaMemcpy(num, g_num, sizeof(long int)*last, cudaMemcpyDeviceToHost);
    cudaMemcpy(den, g_den, sizeof(long int)*last, cudaMemcpyDeviceToHost);

  int count=0;
    //percorre todos elementos comparando-os com todos os outros
    //se o númerador e o denominador são iguais, são mutualmente amigáveis
    for (i = 0; i < last; i++) {
      for (j = i + 1; j < last; j++) {
        if ((num[i] == num[j]) && (den[i] == den[j]))
          count++;
          //printf("%ld and %ld are FRIENDLY\n", sequencia[i], sequencia[j]);
      }
    }
    printf("%d\n", count);
	  gettimeofday(&fim,0);
    printf("\nElapsed time:%f sec\n", (fim.tv_sec+fim.tv_usec/1000000.) - (inic.tv_sec+inic.tv_usec/1000000.));
    
    free(sequencia);
    free(num);
    free(den);

    //Libera a memória alocada na GPU e a reseta
    cudaFree(g_sequencia);
    cudaFree(g_num);
    cudaFree(g_den);
    cudaDeviceReset();
    
	}

	return EXIT_SUCCESS;
}


Overwriting id.cu


In [22]:
!nvcc id.cu -o id && ./id 

1 15000
Number 1 to 15000
293

Elapsed time:0.377938 sec
^C
